# functools: инструменты для манипулирования функциями

Предоставляемые модулем __functools__ инструменты позволяют адаптировать или расширять функции и другие вызываемые объекты без полного их переписывания.

## Декораторы

Основной инструмент, предоставляемый модулем __functools__ — класс __partial__, который можно использовать в качестве “обертки” вокруг вызываемого объекта, задавая значения по умолчанию для части его аргументов. Результирующий объект сам является вызываемым, и c ним можно обращаться так, как если бы это была исходная функция. Он получает те же аргументы, но может вызываться c дополнительными позиционными или именованными аргументами.

Объект типа __partial__ можно использовать вместо __lambda__ для передачи функции аргументов, заданных по умолчанию, в то же время оставляя не указанными некоторые аргументы.

### Объекты partial

В этом примере представлены два простых объекта __partial__ для функции __myfunc()__. Функция __show_details()__ выводит информацию об атрибутах __func__, __args__ и __keywords__ объекта __partial__.


In [11]:
import functools

def myfunc(a, b=2):
    """Docstring for myfunc()."""
    print(' called myfunc with:', (a,b))
        
def show_details(name, f, is_partial=False):
    """Показать детали вызываемого объекта."""
    print('{}:'.format(name))
    print(' object:', f)
    if not is_partial:
        print(' __name__:', f.__name__)
    if is_partial:
        print(' func:', f.func)
        print(' args:', f.args)
        print(' keywords:', f.keywords)
    return
              
show_details('myfunc', myfunc)
myfunc('a',3)
print()
             
# Задать другое значение по умолчанию для 'b', но потребовать, чтобы вызывающий код предоставил 'a'
p1 = functools.partial(myfunc, b=4)
show_details('partial with named default', p1, True)
p1('passing а')
p1('override b', b=5)
print()
   
# Задать значения по умолчанию для ’a' и 'b'
p2 = functools.partial(myfunc, 'default a', b=99)
show_details('partial with defaults', p2, True)
p2()
p2(b='override b')
print()

print('Insufficient arguments:')
p1()

myfunc:
 object: <function myfunc at 0x0000020E76B64F70>
 __name__: myfunc
 called myfunc with: ('a', 3)

partial with named default:
 object: functools.partial(<function myfunc at 0x0000020E76B64F70>, b=4)
 func: <function myfunc at 0x0000020E76B64F70>
 args: ()
 keywords: {'b': 4}
 called myfunc with: ('passing а', 4)
 called myfunc with: ('override b', 5)

partial with defaults:
 object: functools.partial(<function myfunc at 0x0000020E76B64F70>, 'default a', b=99)
 func: <function myfunc at 0x0000020E76B64F70>
 args: ('default a',)
 keywords: {'b': 99}
 called myfunc with: ('default a', 99)
 called myfunc with: ('default a', 'override b')

Insufficient arguments:


TypeError: myfunc() missing 1 required positional argument: 'a'

В конце примера первый из объектов __partial__ вызывается без передачи значения для а, что приводит к возбуждению исключения.

### Копирование и добавление свойств функции

По умолчанию объект __partial__ не имеет атрибутов **__________name__** и **__________doc__**, и их отсутствие затрудняет отладку декорированных функций. Функция **update__wrapper()** позволяет добавлять и копировать атрибуты из исходной функции в объект __partial__.

In [1]:
import functools

def myfunc(a, b=2):
    "Docstring for myfunc()."
    print(' called myfunc with:', (а, b))

def show_details(name, f):
    "Показать детали вызываемого объекта."
    print('{}:'.format(name))
    print(' object:', f)
    print(' __name__:', end=' ')
    try:
        print(f.__name__)
    except AttributeError:
        print('(no __name__)')
    print(' __doc__:', repr(f.__doc__))
    print()
          
show_details('myfunc', myfunc)
          
p1 = functools.partial(myfunc, b=4)
show_details('raw wrapper', p1)
          
print('Updating wrapper:')
print(' assign:', functools.WRAPPER_ASSIGNMENTS)
print(' update:', functools.WRAPPER_UPDATES)
print()
          
functools.update_wrapper(p1, myfunc)
show_details('updated wrapper', p1)

myfunc:
 object: <function myfunc at 0x000002290CEB4550>
 __name__: myfunc
 __doc__: 'Docstring for myfunc().'

raw wrapper:
 object: functools.partial(<function myfunc at 0x000002290CEB4550>, b=4)
 __name__: (no __name__)
 __doc__: 'partial(func, *args, **keywords) - new function with partial application\n    of the given arguments and keywords.\n'

Updating wrapper:
 assign: ('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
 update: ('__dict__',)

updated wrapper:
 object: functools.partial(<function myfunc at 0x000002290CEB4550>, b=4)
 __name__: myfunc
 __doc__: 'Docstring for myfunc().'



Значения по умолчанию атрибутов, добавляемых в функцию-обертку, определяются c помощью константы **WRAPPER_ASSIGNMENTS** уровня модуля, тогда как значения по умолчанию, обновляющие соответствующие атрибуты функции-обертки, — c помощью константы **WRAPPER_UPDATES**.

### Другие вызываемые объекты

Класс __partial__ позволяет работать c любыми вызываемыми объектами, а не только c автономными функциями.

In [3]:
import functools

class MyClass:
    "Демонстрационный класс для functools"

    def __call__(self, e, f=6):
        "Docstring for MyClass.__call__ "
        print(' called object with:', (self, e, f))
      
    
def show_details(name, f):
    "Показать детали вызываемого объекта."
    print('{}:'.format(name))
    print(' object:', f)
    print(' __name__:', end=' ')
    try:
        print(f.__name__)
    except AttributeError:
        print('(no __name__)')
    print(' __doc__', repr(f.__doc__))
    return
      
o = MyClass()
      
show_details('instance', o)
o('e goes here')
print()
             
p = functools.partial(o, e='default for e', f=8)
functools.update_wrapper(p, o)
show_details('instance wrapper', p)
p()

instance:
 object: <__main__.MyClass object at 0x000002290CEDB6D0>
 __name__: (no __name__)
 __doc__ 'Демонстрационный класс для functools'
 called object with: (<__main__.MyClass object at 0x000002290CEDB6D0>, 'e goes here', 6)

instance wrapper:
 object: functools.partial(<__main__.MyClass object at 0x000002290CEDB6D0>, e='default for e', f=8)
 __name__: (no __name__)
 __doc__ 'Демонстрационный класс для functools'
 called object with: (<__main__.MyClass object at 0x000002290CEDB6D0>, 'default for e', 8)


В этом примере объекты __partial__ создаются из экземпляра класса c помощью метода **__________call__()**.

### Методы и функции

В то время как функция __partial()__ возвращает вызываемый объект, готовый к непосредственному использованию, функция __partialmethod()__ возвращает объект, готовый к использованию в качестве несвязанного метода объекта. В следующем примере одна и та же автономная функция добавляется в качестве атрибута класса __MyClass__ дважды: один раз c помощью функции __partialmethod()__, как __method1()__, и второй раз c помощью функции __partial()__, как __method2()__.

In [23]:
import functools

def standalone(self, a=1, b=2):
    "Автономная функция"
    print(' called standalone with:', (self, a, b))
    if self is not None:
        print(' self.attr =', self.attr)
      
    
class MyClass:
    "Демонстрационный класс для functools"
      
    def __init__(self):
        self.attr = 'instance attribute'
      
    method1 = functools.partialmethod(standalone)
    method2 = functools.partial(standalone)
      
o = MyClass()
      
print('standalone')
standalone(None)
print()
      
print('method1 as partialmethod')
o.method1()
print()
      
print('method2 as partial')
try:
    o.method2()
except TypeError as err:
    print('ERROR: {}'.format(err))

standalone
 called standalone with: (None, 1, 2)

method1 as partialmethod
 called standalone with: (<__main__.MyClass object at 0x0000020E76B94A30>, 1, 2)
 self.attr = instance attribute

method2 as partial
ERROR: standalone() missing 1 required positional argument: 'self'


### Наделение декоратора свойствами декорируемой функции

Обновление свойств функции-обертки особенно полезно в случае декораторов, поскольку это позволяет сделать ее похожей на исходную “голую” функцию.

In [29]:
import functools

def show_details(name, f):
    "Показать детали вызываемого объекта."
    print('{}:'.format(name))
    print(' object:', f)
    print(' __name__:', end=' ')
    try:
        print (f.__name__)
    except AttributeError:
        print('(no __name__)')
    print(' __doc__', repr(f.__doc__))
    print()
      
def simple_decorator(f):
    @functools.wraps(f)
    def decorated(a='decorated defaults', b=1):
        print(' decorated:', (a, b))
        print(' ', end=' ')
        return f(a, b=b)
    return decorated
      
def myfunc(a, b=2) :
    "myfunc() is not complicated"
    print(' myfunc:', (a, b))
    return
      
# Исходная функция
show_details('myfunc', myfunc)
myfunc('unwrapped, default b')
myfunc('unwrapped, passing b', 3)
print()
      
# Явное упаковывание
wrapped_myfunc = simple_decorator(myfunc)
show_details('wrapped_myfunc', wrapped_myfunc)
wrapped_myfunc()
wrapped_myfunc('args to wrapped', 4)
print()
             
# Упаковывание c помощью синтаксиса декоратора
@simple_decorator
def decorated_myfunc(a, b):
    myfunc(a, b)
    return
             
show_details('decorated_myfunc', decorated_myfunc)
decorated_myfunc()
decorated_myfunc('args to decorated', 4)

myfunc:
 object: <function myfunc at 0x0000020E76C64A60>
 __name__: myfunc
 __doc__ 'myfunc() is not complicated'

 myfunc: ('unwrapped, default b', 2)
 myfunc: ('unwrapped, passing b', 3)

wrapped_myfunc:
 object: <function myfunc at 0x0000020E76B64550>
 __name__: myfunc
 __doc__ 'myfunc() is not complicated'

 decorated: ('decorated defaults', 1)
   myfunc: ('decorated defaults', 1)
 decorated: ('args to wrapped', 4)
   myfunc: ('args to wrapped', 4)

decorated_myfunc:
 object: <function decorated_myfunc at 0x0000020E76C55CA0>
 __name__: decorated_myfunc
 __doc__ None

 decorated: ('decorated defaults', 1)
   myfunc: ('decorated defaults', 1)
 decorated: ('args to decorated', 4)
   myfunc: ('args to decorated', 4)


Модуль __functools__ предоставляет декоратор __wraps()__, который применяет функцию __update_wrapper()__ к декорируемой функции.

## Сравнение

B ***Python 2*** классы могут определять метод **__________cmp__()**, который возвращает значение -1, 0 или 1, в зависимости от того, является ли объект соответственно меньшим, равным или большим, чем элемент, c которым он сравнивается.

В ***Python 2.1*** появился API методов расширенного сравнения (**__________lt__()**, **__________le__()**, **__________eq__()**, **__________ne__()**, **__________gt__()** и **__________ge__()**), которые выполняют сравнение и возвращают булево значение. 

В ***Python 3*** от метода **__________cmp__()** отказались как от устаревшего в пользу этих новых методов, и модуль __functools__ предоставляет инструменты, которые упрощают написание классов, совместимых c требованиями к операциям сравнения в ***Python 3***.

### Расширенное сравнение

API расширенного сравнения проектировался c таким расчетом, чтобы обеспечить максимально эффективную реализацию каждого теста для классов со сложными операциями сравнения. Однако в случае классов c относительно простыми операциями сравнения не имеет смысла создавать вручную каждый метод.

Декоратор классов __total_ordering()__ получает класс, который предоставляет некоторые из методов сравнения, и добавляет остальные методы.

In [32]:
import functools
import inspect
from pprint import pprint

@functools.total_ordering
class MyObject:
    
    def __init__ (self, val):
        self.val = val

    def __eq__(self, other):
        print(' testing__eq__({}, {})'.format(self.val, other.val))
        return self.val == other.val

    def __gt__(self, other):
        print(' testing__gt__({}, {})'.format(self.val, other.val))
        return self.val > other.val

print('Methods:\n')
pprint(inspect.getmembers(MyObject, inspect.isfunction))
      
a = MyObject(1)
b = MyObject(2)
      
print('\nComparisons:')
for expr in ['a < b', 'a <= b', 'a == b', 'a >= b', 'a > b']:
    print('\n{:<6}:'.format(expr))
    result = eval(expr)
    print(' result of {}: {}'.format(expr, result))

Methods:

[('__eq__', <function MyObject.__eq__ at 0x0000020E76C55F70>),
 ('__ge__', <function _ge_from_gt at 0x0000020E72D638B0>),
 ('__gt__', <function MyObject.__gt__ at 0x0000020E76C554C0>),
 ('__init__', <function MyObject.__init__ at 0x0000020E76C553A0>),
 ('__le__', <function _le_from_gt at 0x0000020E72D63940>),
 ('__lt__', <function _lt_from_gt at 0x0000020E72D63820>)]

Comparisons:

a < b :
 testing__gt__(1, 2)
 testing__eq__(1, 2)
 result of a < b: True

a <= b:
 testing__gt__(1, 2)
 result of a <= b: True

a == b:
 testing__eq__(1, 2)
 result of a == b: False

a >= b:
 testing__gt__(1, 2)
 testing__eq__(1, 2)
 result of a >= b: False

a > b :
 testing__gt__(1, 2)
 result of a > b: False


Класс должен предоставить реализацию метода **__________eq__()** и одного из оставшихся методов расширенного сравнения. Декоратор добавляет реализации остальных методов, которые работают, используя предоставленные реализации. Если сравнение не может быть выполнено, метод должен вернуть значение __NotImplemented__, чтобы можно было попытаться использовать операторы обратного сравнения c другим объектом, прежде чем признать операцию неудачной.

### Порядок сортировки

Поскольку функции сравнения старого стиля признаны устаревшими в ___Python 3___, аргумент __cmp__ в функциях наподобие __sort()__ больше не поддерживается.

Функция __cmp_to_key()__ обеспечивает преобразование старых функций в функции, которые возвращают ключ сортировки, определяющий позицию объекта в конечной последовательности.

In [3]:
import functools

class MyObject:
    
    def __init__(self, val):
        self.val = val

    def __str__(self):
        return 'MyObject({})'.format(self.val)

def compare_obj(a, b):
    """Функция сравнения старого стиля."""
    print('comparing {} and {}'.format(a,b))
    if a.val < b.val:
        return -1
    elif a.val > b.val:
        return 1
    return 0
      
# Заставить функцию ключа использовать функцию cmp_to_key()
get_key = functools.cmp_to_key(compare_obj)
      
def get_key_wrapper(o):
    "Функция-обертка для get_key, разрешающая инструкции вывода."
    new_key = get_key(o)
    print('key_wrapper({}) -> {!r}'.format(o, new_key))
    return new_key
      
objs = [MyObject(x) for x in range(5, 0, -1)]
      
for o in sorted(objs, key=get_key_wrapper):
    print(o)

key_wrapper(MyObject(5)) -> <functools.KeyWrapper object at 0x00000152877013B0>
key_wrapper(MyObject(4)) -> <functools.KeyWrapper object at 0x0000015287701690>
key_wrapper(MyObject(3)) -> <functools.KeyWrapper object at 0x00000152877010D0>
key_wrapper(MyObject(2)) -> <functools.KeyWrapper object at 0x00000152877012D0>
key_wrapper(MyObject(1)) -> <functools.KeyWrapper object at 0x0000015287701210>
comparing MyObject(4) and MyObject(5)
comparing MyObject(3) and MyObject(4)
comparing MyObject(2) and MyObject(3)
comparing MyObject(1) and MyObject(2)
MyObject(1)
MyObject(2)
MyObject(3)
MyObject(4)
MyObject(5)


Обычно функцию __cmp_to_key()__ используют непосредственно, но в данном примере привлекается функция-обертка, обеспечивающая вывод дополнительной информации при вызове функции сопоставления.

Как следует из этих результатов, выполнение функции __sorted()__ начинается c вызова функции __get_key_wrapper()__ для каждого элемента последовательности c целью генерации ключей. Ключами, возвращаемыми функцией __cmp_to_key()__, являются экземпляры определенного в модуле __functools__ класса, который реализует API расширенного сравнения c использованием передаваемой ему функции сравнения старого стиля. Созданные ключи используются для сортировки последовательности путем их сравнения.

## Кеширование

Декоратор __lru_cache()__ обертывает функцию кешем LRU (от англ, teast recently used — “вытесняется элемент, неиспользованный дольше всех”). Аргументы функции используются для создания хеш-значения, которое сопоставляется c результатом. Последующие вызовы функции c теми же аргументами будут заменяться извлечением соответствующего значения из кеша. Кроме того, этот декоратор добавляет в функцию методы, обеспечивающие проверку состояния (__cache_info()__) и очистку (__cache_clear()__) кеша.

In [8]:
import functools

@functools.lru_cache()
def expensive(a, b):
    print('expensive({}, {})'.format(a, b))
    return a * b
      
MAX = 2
      
print('First set of calls:')
for i in range(MAX):
    for j in range(MAX):
        expensive(i, j)
print(expensive.cache_info())
      
print('\nSecond set of calls:')
for i in range(MAX + 1):
    for j in range(MAX + 1):
        expensive(i, j)
print(expensive.cache_info())
      
print('\nClearing cache:')
expensive.cache_clear()
print(expensive.cache_info())
      
print('\nThird set of calls:')
for i in range(MAX):
    for j in range(MAX):
        expensive(i, j)
print(expensive.cache_info())

First set of calls:
expensive(0, 0)
expensive(0, 1)
expensive(1, 0)
expensive(1, 1)
CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)

Second set of calls:
expensive(0, 2)
expensive(1, 2)
expensive(2, 0)
expensive(2, 1)
expensive(2, 2)
CacheInfo(hits=4, misses=9, maxsize=128, currsize=9)

Clearing cache:
CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

Third set of calls:
expensive(0, 0)
expensive(0, 1)
expensive(1, 0)
expensive(1, 1)
CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)


В этом примере выполняется серия вызовов функции __expensive()__ в наборе вложенных циклов. При повторных вызовах функции c одними и теми же значениями аргументов результаты появляются в кеше. При повторном выполнении циклов после очистки кеша значения должны вычисляться заново.

Неограниченный рост размера кеша в длительно выполняющихся процессах можно предотвратить, задав его максимальный размер. Размер по умолчанию —
128 записей, но его можно регулировать для каждого кеша c помощью аргумента __maxsize__.

In [6]:
import functools

@functools.lru_cache()
def expensive(a, b):
    print('expensive({}, {})'.format(a, b))
    return a * b

def make_call(a, b):
    print('({}, {})'.format(a, b), end=' ')
    pre_hits = expensive.cache_info().hits
    expensive(a, b)
    post_hits = expensive.cache_info().hits
    if post_hits > pre_hits:
        print('cache hit')
             
print('Establish the cache')
make_call(1, 2)
make_call(2, 3)
      
print('\nUse cached items')
make_call(1, 2)
make_call(2, 3)
      
print('\nCompute a new value, triggering cache expiration')
make_call(3, 4)
      
print('\nCache still contains one old item')
make_call(2, 3)
      
print('\nOldest item needs to be recomputed')
make_call(1, 2)

Establish the cache
(1, 2) expensive(1, 2)
(2, 3) expensive(2, 3)

Use cached items
(1, 2) cache hit
(2, 3) cache hit

Compute a new value, triggering cache expiration
(3, 4) expensive(3, 4)

Cache still contains one old item
(2, 3) cache hit

Oldest item needs to be recomputed
(1, 2) cache hit


В этом примере размер кеша ограничен двумя записями. При использовании третьего наборауникальных аргументов (3,4) самая давняя из записей заменяется новым результатом.

Ключи для кеша, управляемого функцией __lru_cache()__, должны быть хешируемыми, и поэтому такими же должны быть аргументы, передаваемые функции, которая обернута кешем.

In [14]:
import functools


@functools.lru_cache(maxsize=2)
def expensive(a, b):
    print('called expensive({}, {})'.format(a, b))
    return a * b


def make_call(a, b):
    print('({}, {})'.format(a, b), end=' ')
    pre_hits = expensive.cache_info().hits
    expensive(a, b)
    post_hits = expensive.cache_info().hits
    if post_hits > pre_hits:
        print('cache hit')


make_call(1, 2)

try:
    make_call([1], 2)
except TypeError as err:
    print('ERROR: {}'.format(err))

try:
    make_call(1, {'2': 'two'})
except TypeError as err:
    print('ERROR: {}'.format(err))

(1, 2) called expensive(1, 2)
([1], 2) ERROR: unhashable type: 'list'
(1, {'2': 'two'}) ERROR: unhashable type: 'dict'


Если функции передается объект, который нельзя хешировать, возбуждается исключение __TypeError__.

## Редукция набора данных

Функция __reduce()__ получает вызываемый объект и последовательность данных в качестве аргументов. Результатом ее работы является единственное значение, основанное на вызове объекта со значениями из последовательности и накоплении результатов.

In [15]:
import functools


def do_reduce(a, b):
    print('do_reduce({}, {})'.format(a, b))
    return a + b


data = range(1, 5)
print(data)
result = functools.reduce(do_reduce, data)
print('result: {}'.format(result))

range(1, 5)
do_reduce(1, 2)
do_reduce(3, 3)
do_reduce(6, 4)
result: 10


В этом примере суммируются числа, образующие входную последовательность.

Необязательный аргумент __initializer__ помещается перед последовательностью и обрабатывается наряду c другими элементами. Это может быть использовано для обновления ранее вычисленного значения новыми входными данными.

In [7]:
import functools


def do_reduce(a, b):
    print('do_reduce({}, {})'.format(a, b))
    return a + b


data = range(1, 5)
print(data)
result = functools.reduce(do_reduce, data, 99)
print('result: {}'.format(result))

range(1, 5)
do_reduce(99, 1)
do_reduce(100, 2)
do_reduce(102, 3)
do_reduce(105, 4)
result: 109


В этом примере ранее найденная сумма, равная 99, используется для инициализации значения, вычисляемого функцией __reduce()__.

Последовательности c единственным элементом редуцируются до этого значения, если не предоставлен аргумент __initializer__. В отсутствие этого аргумента пустые списки генерируют ошибку.

In [8]:
import functools


def do_reduce(a, b):
    print('do_reduce({}, {})'.format(a, b))
    return a + b


print('Single item in sequence:',
      functools.reduce(do_reduce, [1]))

print('Single item in sequence with initializer:',
      functools.reduce(do_reduce, [1], 99))

print('Empty sequence with initializer:',
      functools.reduce(do_reduce, [], 99))

try:
    print('Empty sequence:', functools.reduce(do_reduce, []))
except TypeError as err:
    print('ERROR: {}'.format(err))

Single item in sequence: 1
do_reduce(99, 1)
Single item in sequence with initializer: 100
Empty sequence with initializer: 99
ERROR: reduce() of empty sequence with no initial value


Поскольку аргумент __initializer__ выступает в качестве значения по умолчанию и при этом сочетается c новыми значениями, если входная последовательность не является пустой, очень важно тщательно проверять, используется ли он надлежащим образом. Если в сочетании значения по умолчанию c новыми значениями нет смысла, лучше перехватывать исключение __TypeError__, чем передавать аргумент __initializer__.

## Обобщенные функции

В языках программирования c динамической типизацией, таких как ***Python***, часто возникает необходимость в изменении характера выполнения операций в зависимости от типа аргументов, особенно если речь идет о проведении различий между списком элементов и одиночным элементом. Непосредственная проверка типов не вызывает затруднений, однако в тех случаях, когда различия в поведении могут быть вынесены в отдельные функции, модуль __functools__ предоставляет декоратор __singledispatch()__, который обеспечивает регистрацию набора обобщенных функций c возможностью автоматического выбора нужной функции на основании типа первого аргумента.

In [17]:
import functools

@functools.singledispatch
def myfunc(arg):
    print('default myfunc({!r})'.format(arg))


@myfunc.register(int)
def myfunc_int(arg):
    print('myfunc_int({})'.format(arg))


@myfunc.register(list)
def myfunc_list(arg):
    print('myfunc_list()')
    for item in arg:
        print('  {}'.format(item))


myfunc('string argument')
myfunc(1)
myfunc(2.3)
myfunc(['a', 'b', 'c'])

default myfunc('string argument')
myfunc_int(1)
default myfunc(2.3)
myfunc_list()
  a
  b
  c


Атрибут __register()__ новой функции служит дополнительным декоратором для регистрации альтернативной реализации. Первая функция, обернутая декоратором __singledispatch()__, является реализацией по умолчанию, если не обнаружена никакая другая функция, специфическая для данного типа, как в случае типа __float__ в этом примере.

Если не удается найти точного соответствия типу, определяется порядок наследования и используется наиболее соответствующий тип.

In [18]:
import functools


class A:
    pass


class B(A):
    pass


class C(A):
    pass


class D(B):
    pass


class E(C, D):
    pass


@functools.singledispatch
def myfunc(arg):
    print('default myfunc({})'.format(arg.__class__.__name__))


@myfunc.register(A)
def myfunc_A(arg):
    print('myfunc_A({})'.format(arg.__class__.__name__))


@myfunc.register(B)
def myfunc_B(arg):
    print('myfunc_B({})'.format(arg.__class__.__name__))


@myfunc.register(C)
def myfunc_C(arg):
    print('myfunc_C({})'.format(arg.__class__.__name__))


myfunc(A())
myfunc(B())
myfunc(C())
myfunc(D())
myfunc(E())

myfunc_A(A)
myfunc_B(B)
myfunc_C(C)
myfunc_B(D)
myfunc_C(E)


В этом примере для классов D и E не находится точного соответствия c какой-либо зарегистрированной обобщенной функцией, и выбор функции зависит от иерархии классов.

## Задания